In [11]:
import pandas as pd
import json

In [17]:
with open('dwts_scraper/scores.json') as f:
    dwts = json.load(f)

In [53]:
df_dwts = pd.read_json('dwts_scraper/scores.json')

In [54]:
df_dwts['judges'] = df_dwts['judge_phrase'].str.split(",", n = 1, expand = False)

In [26]:
df_dwts['scores']

0          18 (7, 5, 6)
1          19 (6, 7, 6)
2          18 (6, 6, 6)
3          22 (7, 8, 7)
4          21 (7, 7, 7)
             ...       
2886    30 (10, 10, 10)
2887    30 (10, 10, 10)
2888    30 (10, 10, 10)
2889    30 (10, 10, 10)
2890    30 (10, 10, 10)
Name: scores, Length: 2891, dtype: object

In [55]:
new_cols = df_dwts['scores'].str.extract(r'(?P<total_score>\d+)\s\((?P<indiv_scores>.*)\)$')
new_cols['judge_score_list'] = new_cols['indiv_scores'].str.split(",")
new_cols

,total_score,indiv_scores,judge_score_list
0,18,"7, 5, 6","[7, 5, 6]"
1,19,"6, 7, 6","[6, 7, 6]"
2,18,"6, 6, 6","[6, 6, 6]"
3,22,"7, 8, 7","[7, 8, 7]"
4,21,"7, 7, 7","[7, 7, 7]"
...,...,...,...
2886,30,"10, 10, 10","[10, 10, 10]"
2887,30,"10, 10, 10","[10, 10, 10]"
2888,30,"10, 10, 10","[10, 10, 10]"
2889,30,"10, 10, 10","[10, 10, 10]"


In [56]:
df_dwts_new = pd.concat([df_dwts, new_cols], axis = 1)

In [57]:
df_dwts_new.head(1)

,couple,scores,dance,music,result,season,week,judge_phrase,tv_show,score,...,grease_music,horror_film_show,queen_music,unnamed_5,janet_jackson_music,villain,judges,total_score,indiv_scores,judge_score_list
0,George & Edyta,"18 (7, 5, 6)",Cha-cha-cha,"""Oye Como Va""—Santana",Safe,2,Week 1,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonioli.]",18,"7, 5, 6","[7, 5, 6]"


In [58]:
df_dwts_new.head(1).explode(['judges','judge_score_list'])

ValueError: columns must have matching element counts